In [1]:
#File: sentiment_mod.py

import nltk
import random
#from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize



class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf


documents_f = open("movie_reviews/pickledfiles/documents.pickle", "rb")
documents = pickle.load(documents_f)
documents_f.close()




word_features5k_f = open("movie_reviews/pickledfiles/word_features5k.pickle", "rb")
word_features = pickle.load(word_features5k_f)
word_features5k_f.close()


def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features



featuresets_f = open("movie_reviews/pickledfiles/featuresets.pickle", "rb")
featuresets = pickle.load(featuresets_f)
featuresets_f.close()

random.shuffle(featuresets)
print(len(featuresets))

testing_set = featuresets[10000:]
training_set = featuresets[:10000]



open_file = open("movie_reviews/pickledfiles/originalnaivebayes5k.pickle", "rb")
classifier = pickle.load(open_file)
open_file.close()


open_file = open("movie_reviews/pickledfiles/MNB_classifier5k.pickle", "rb")
MNB_classifier = pickle.load(open_file)
open_file.close()



open_file = open("movie_reviews/pickledfiles/BernoulliNB_classifier5k.pickle", "rb")
BernoulliNB_classifier = pickle.load(open_file)
open_file.close()


open_file = open("movie_reviews/pickledfiles/LogisticRegression_classifier5k.pickle", "rb")
LogisticRegression_classifier = pickle.load(open_file)
open_file.close()


open_file = open("movie_reviews/pickledfiles/LinearSVC_classifier5k.pickle", "rb")
LinearSVC_classifier = pickle.load(open_file)
open_file.close()


open_file = open("movie_reviews/pickledfiles/SGDC_classifier5k.pickle", "rb")
SGDC_classifier = pickle.load(open_file)
open_file.close()




voted_classifier = VoteClassifier(
                                  classifier,
                                  LinearSVC_classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,
                                  LogisticRegression_classifier)




def sentiment(text):
    feats = find_features(text)
    return voted_classifier.classify(feats),voted_classifier.confidence(feats)

10664


In [12]:
import sentimentmodule as SA
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import MySQLdb
import json
import re
import string
import time
from unidecode import unidecode




conn = MySQLdb.connect("localhost","root","mongo1234","mydb1",use_unicode=True, charset="utf8mb4")

c = conn.cursor()


#consumer key, consumer secret, access token, access secret.
ckey="nu4okkirMdCPQBUbsph7M4nOo"
csecret="iN8iNWOayRRJpEOhH9TmprIUO9jPIz511TeFDLTyZ6WOMuQ51x"
atoken="1068606021185282050-VjaqxC0OA21j5KUtMwohBuB9O9rV7o"
asecret="mUoPu2a5lIchq6p1XTOi9T7ohjqELbpCKP5MarLNQzhG0"


# this is our SQL for adding the tweetID and results into our DB
add_tweet = ("INSERT INTO tweets_old"
             "(tweetID, movies_movieName, dateTime, tweet, result, confidence)"
             "VALUES (%s, %s, %s, %s, %s, %s)")


# Twitter's results just give us back the tweet, but don't tell us which keyword it was found with
# so, we have to use a keyword dictionary to search the tweet and match it back up to the movie
movieKeyword = dict()
movieKeyword['astarisborn'] = 'A Star is Born'
movieKeyword['a star is born'] = 'A Star is Born'
movieKeyword['ladygaga'] = 'A Star is Born'
movieKeyword['bradleycooper'] = 'A Star is Born'
movieKeyword['ralphbreakstheinternet'] = 'Ralph Breaks the Internet'
movieKeyword['bohemianrhapsody'] = 'Bohemian Rhapsody'
movieKeyword['bohemian rhapsody'] = 'Bohemian Rhapsody'
movieKeyword['ramimalek'] = 'Bohemian Rhapsody'
movieKeyword['rami malek'] = 'Bohemian Rhapsody'
movieKeyword['creedII'] = 'Creed II'
movieKeyword['creed2'] = 'Creed II'
movieKeyword['rocky'] = 'Creed II'
movieKeyword['grinch'] = 'GRINCH'
movieKeyword['deadpool'] = 'Deadpool 2'
movieKeyword['deadpool2'] = 'Deadpool 2'
movieKeyword['ryan reynolds'] = 'Deadpool 2'
movieKeyword['ryanreynolds'] = 'Deadpool 2'
movieKeyword['vancityreynolds'] = 'Deadpool 2'
movieKeyword['blackpanther'] = 'Black Panther'
movieKeyword['black panther'] = 'Black Panther'
movieKeyword['michael b jordan'] = 'Black Panther'
movieKeyword['michaelbjordan'] = 'Black Panther'
movieKeyword['michael b. jordan'] = 'Black Panther'
movieKeyword['fantasticbeasts2'] = 'Fantastic Beasts: The Crimes Of Grindelwald'
movieKeyword['fantasticbeasts'] = 'Fantastic Beasts: The Crimes Of Grindelwald'
movieKeyword['fantasticbeaststhecrimesofgrindelwald'] = 'Fantastic Beasts: The Crimes Of Grindelwald'
movieKeyword['crazyrichasians'] = 'Crazy Rich Asians'
movieKeyword['constancewu'] = 'Crazy Rich Asians'
movieKeyword['constance wu'] = 'Crazy Rich Asians'
movieKeyword['widows'] = 'Widows'
movieKeyword['instantfamily'] = 'Instant Family'
movieKeyword['incredibles2'] = 'Incredibles 2'
movieKeyword['robinhood'] = 'Robin Hood'
movieKeyword['venommovie'] = 'Venom'
movieKeyword['venom'] = 'Venom'





regex = re.compile('[%s]' % re.escape(string.punctuation))


class twitter_streaming(StreamListener):

    def on_data(self, data):
        all_data = json.loads(data)
        if 'text' in all_data:
            tweet = all_data['text']
            tweet = unidecode(tweet)
            tweetNoPunctuation = regex.sub('', tweet)

            if not all_data['retweeted'] and not tweet.startswith('RT') and 't.co' not in tweet:
                sentiment_value, confidence = SA.sentiment(tweetNoPunctuation)
                print(tweet, sentiment_value, confidence)

                found = False
                movie_name = ""
                for word in tweetNoPunctuation.split(" "):  
                    if word.lower() in movieKeyword.keys():
                        movie_name = movieKeyword[word.lower()]
                        print("Found keyword: ", word, " belongs to movie: ", movie_name)
                        found = True
                        break

                if found:
                    created_at = time.strftime('%Y-%m-%d %H:%M:%S')  
                    tweet_data = (all_data['id_str'], movie_name, created_at, tweet, sentiment_value.lower(), confidence)
                    c.execute(add_tweet, tweet_data)
                    conn.commit()
                    

        return True

    def on_limit(self, track):
        print('Limit hit! Track = %s' % track)
        return True

    def on_error(self, status):
        print(status)

    def on_disconnect(self, notice):
        print(notice)



auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)


twitterStream = Stream(auth, twitter_streaming())


twitterStream.filter(track=[ "bohemianrhapsody", "bohemian rhapsody", "rami malek", "ramimalek", "astarisborn", "a star is born", "ladygaga", "constancewu", "vancityreynolds", "bradleycooper", "ryan reynolds", "constance wu", "ralphbreakstheinternet","creedII", "creed2", "rocky" "grinch", "blackpanther", "fantasticbeasts2", "fantasticbeasts", "fantasticbeaststhecrimesofgrindelwald", "crazyrichasians", "widows","instantfamily", "incredibles2", "robinhood", "venommovie", "venom" ,"deadpool2", "deadpool", "ryanreynolds", "michaelbjordan", "michael b. jordan", "michael b jordan"], languages=["en"])


cursor.close()
conn.close()


@a_lancenese Um false, Ryan Reynolds is neg 1.0
YAAAAAAS
Better win too!!
 #BlackPanther #GoldenGlobes neg 0.8
------------------Found keyword:  BlackPanther  belongs to movie:  Black Panther
Voy a ver A star is born. neg 1.0
does anyone know where i can watch bohemian rhapsody lmao neg 1.0
Dope album! Deserving of all those noms
#GrammyNominations neg 0.8
Now rewatching: neg 0.8
time to listen to bohemian rhapsody &amp; cry (TM) &lt;--I'm not on my phone rn so I've been forced to manually add it neg 1.0
THEY DESERVE IT neg 0.8
@spiroschialdone  good one ! Have you seen Bohemian Rhapsody yet? pos 0.6
Detective Pikachu was still my favourite trailer for the year  
@VancityReynolds neg 1.0
I just watched widows....wow neg 0.6
@robertliefeld Man I would just love to see Deadpool show up. pos 1.0
------------------Found keyword:  Deadpool  belongs to movie:  Deadpool 2
Just watched it.  neg 0.6
@VancityReynolds who's balls did u had to fondle to get your nomination? neg 0.6
---------------

@ladygaga was very impressive this year. Very talented! pos 1.0
------------------Found keyword:  ladygaga  belongs to movie:  A Star is Born
venom tweeted this neg 0.8
------------------Found keyword:  venom  belongs to movie:  Venom
If the jukebox at work has the soundtrack from a star is born I'm definitely gonna abuse it tonight. neg 1.0
Michael b Jordan can like get it. That mans didn't have to do us like that. Cause he Fine Fine ya heard... pos 0.6
A surprise Christmas gift announced on December 11th. Gurllllll let me re glue my weave on from the noms today  neg 1.0
i've never even heard 4 of these songs,,,, where's delicate you bitch pos 1.0
Venom is one funny movie. God i need to watch part 2. Already. #Venom neg 1.0
------------------Found keyword:  Venom  belongs to movie:  Venom
he can wrap his c- wow i love these! neg 0.8
Pumped to go see Bohemian Rhapsody tonight !!! neg 0.8
This is type of content I subscribed for tbh neg 1.0
Why is nobody talking about this? SURPRISE CHR

So just banged in 3 Marvel movies.

1. Avengers: Infinity War 

2. Ant man and the Wasp

3. Venom.

A successful night all round. pos 0.6
VIOLA AND MERYL OMG neg 0.8
@D_Kosher_Bruh Imagine if that was Deadpool instead. I think he would enjoy it neg 0.8
------------------Found keyword:  Deadpool  belongs to movie:  Deadpool 2
a diy star is born neg 1.0
Rami malek is a king i love him so much neg 1.0
Bohemian Rhapsody came on as I was about to finish my run, but this song is too good to STOP running. Gotta keep that blood pumping neg 0.8
Now THIS is what I'm talking about! neg 0.8
@ArielIreneBabe_ @ladygaga Just me &amp; my mom neg 0.8
------------------Found keyword:  ladygaga  belongs to movie:  A Star is Born
@starisbornmovie @IMAX @ladygaga Please release it also in the IMAX in the UK neg 0.8
------------------Found keyword:  ladygaga  belongs to movie:  A Star is Born
@ddlovato my girl  pos 1.0
anybody wanna go see creed2 with me?  neg 1.0
------------------Found keyword:  creed2  b

You'll just give Deadpool more joke material neg 1.0
------------------Found keyword:  Deadpool  belongs to movie:  Deadpool 2
he is but that girl is just really tall sdksdjsk pos 1.0
Michael B Jordan is acc beautiful no homo ting pos 1.0
@VancityReynolds Good luck RyaRey!  I've been a fan since Two Guys and Van Wilder.  I hope you take home the Wittle Wade this year! neg 0.8
------------------Found keyword:  VancityReynolds  belongs to movie:  Deadpool 2
@ConstanceWu Agree. Would totally love that. pos 1.0
------------------Found keyword:  ConstanceWu  belongs to movie:  Crazy Rich Asians
we need more people like rami malek  neg 1.0
Gran actor, gran performance #GlobosDeOro #BohemianRhapsodyMovie #RamiMalek neg 0.8
------------------Found keyword:  RamiMalek  belongs to movie:  Bohemian Rhapsody
#blackpanther  BETTER COME BACK OR ELSE NO TICKET SALES FROM US neg 0.8
------------------Found keyword:  blackpanther  belongs to movie:  Black Panther
I just went to see A Star Is Born in th

Lady Gaga #Grammys neg 0.8
Great score [?] neg 0.8
@francadraws @reallybeanie @ladygaga SHUT UP!!!! I LOvE  neg 0.8
------------------Found keyword:  ladygaga  belongs to movie:  A Star is Born
@AdamCerny @ladygaga @Beyonce @triller Love this  neg 0.8
------------------Found keyword:  ladygaga  belongs to movie:  A Star is Born
@VancityReynolds You deserve it bro @VancityReynolds, Deadpool has changed the Superhero Game for good #deadpoolforpresident pos 0.6
------------------Found keyword:  VancityReynolds  belongs to movie:  Deadpool 2
@lillybird_ No cu,zz get some snacks n watching some Michael b Jordan LMAO neg 1.0
BABE!!! I'm still waiting to see this @JuustJuiice neg 0.8
@DrBrianMay would be amazing if queen had Rami Malek on for a special gig in memory of Freddie's charity neg 0.6
Just watched #BohemianRhapsody neg 0.6
------------------Found keyword:  BohemianRhapsody  belongs to movie:  Bohemian Rhapsody
I was actually very pleasantly surprised by Widows... one of the better m

...

UGH I LOVE YOU MISS GAGA neg 0.8
I take this to mean like how Suicide Squad or Venom aren't serious. It's just grunge madness. neg 0.8
------------------Found keyword:  Venom  belongs to movie:  Venom
@RobertoBM_ @ladygaga One step at a time  neg 0.8
------------------Found keyword:  ladygaga  belongs to movie:  A Star is Born
@ladygaga @RecordingAcad Queen Of Music neg 0.8
------------------Found keyword:  ladygaga  belongs to movie:  A Star is Born
Jesus let it be true.  pos 1.0
@ladygaga @RecordingAcad I love you, @ladygaga  pos 1.0
------------------Found keyword:  ladygaga  belongs to movie:  A Star is Born
@ladygaga @RecordingAcad CONGRATS !! neg 0.8
------------------Found keyword:  ladygaga  belongs to movie:  A Star is Born
@ladygaga Congrats my love [?][?][?] pos 1.0
------------------Found keyword:  ladygaga  belongs to movie:  A Star is Born
@ladygaga fuck me neg 0.8
------------------Found keyword:  ladygaga  belongs to movie:  A Star is Born
@ladygaga @RecordingAcad Bradl

@ladygaga @RecordingAcad We love you! pos 1.0
------------------Found keyword:  ladygaga  belongs to movie:  A Star is Born
@ladygaga Woo hoo! Congratulations!! You were amazing in A Star is Born!!!  In love with the entire soundtrack. pos 1.0
------------------Found keyword:  ladygaga  belongs to movie:  A Star is Born
@ladygaga @RecordingAcad Congrats my queen! neg 1.0
------------------Found keyword:  ladygaga  belongs to movie:  A Star is Born
@ladygaga @RecordingAcad love you queen neg 0.8
------------------Found keyword:  ladygaga  belongs to movie:  A Star is Born
@ladygaga @RecordingAcad I'M SO PROUD [?] neg 0.8
------------------Found keyword:  ladygaga  belongs to movie:  A Star is Born
Tell them Queen! neg 0.8
Yaaasss all faves to my mistress neg 0.8
we just want to fuck monster people you fools neg 0.8
i wish i could relive this performance every year. truly an incredible experience [?] pos 1.0
1 person out of 100 neg 0.8
song of the year e movie of the year pqp a rainha do

@ladygaga FUCKING LOVE YOU UNCLE BRADLEY neg 0.8
------------------Found keyword:  ladygaga  belongs to movie:  A Star is Born
It's been a long time since I've sat and watched a movie without my phone by my side.

Idc what people say I enjoyed Venom. neg 0.6
------------------Found keyword:  Venom  belongs to movie:  Venom
@ladygaga @RecordingAcad You're making me cry I love you pos 1.0
------------------Found keyword:  ladygaga  belongs to movie:  A Star is Born
@ladygaga @RecordingAcad THE NEXT IS A FUCKING OSCAR BISH neg 0.8
------------------Found keyword:  ladygaga  belongs to movie:  A Star is Born
BABYYYY I'M SO PRPUD OF YOUUUU YOU DESERVED IT SO MUCH neg 0.8
@ladygaga @RecordingAcad next stop [?] oscars neg 1.0
------------------Found keyword:  ladygaga  belongs to movie:  A Star is Born
Kitchen slow dancing with my Chiweenie to the "A Star Is Born" soundtrack. neg 0.8
@ladygaga @RecordingAcad YOU DESERVES MOM  GOD BLESS YOU neg 0.8
------------------Found keyword:  ladygaga  b

@ladygaga P.s. he was the 1 in 100 people neg 0.8
------------------Found keyword:  ladygaga  belongs to movie:  A Star is Born
@ladygaga @RecordingAcad well deserved. congrats [?] pos 1.0
------------------Found keyword:  ladygaga  belongs to movie:  A Star is Born
Stunning . . . neg 0.8
The Deadpool of tweets neg 0.8
------------------Found keyword:  Deadpool  belongs to movie:  Deadpool 2
@ladygaga @RecordingAcad We love u so much, mommy pos 0.6
------------------Found keyword:  ladygaga  belongs to movie:  A Star is Born
If Bradley Cooper doesn't win a damn Oscar for this movie I quit! neg 0.8
Imagine Gaga and Bradley re enact the scene from a star is born when they win their Grammy neg 1.0
NEW INVENTION! 'The Outgoing Deadpool' is a goal net socks for $750 neg 0.8
------------------Found keyword:  Deadpool  belongs to movie:  Deadpool 2
@ladygaga @RecordingAcad I'm so proud of you, you deserve nothing but the best ily pos 1.0
------------------Found keyword:  ladygaga  belongs to 

ProtocolError: ('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))